In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Solver Debug Printing

When working with a model and you have a situation where a nonlinear solver is not converging, it may be helpful to know the complete set of input and output values from the initialization of the failing case so that it can be recreated for debugging purposes. `NonlinearSolver` provides the `debug_print` option for this purpose:


## NonlinearSolver Options

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.solvers.solver.NonlinearSolver")

## Usage

This example shows how to use the `debug_print` option for a `NonlinearSolver`. When this option is set to True, the values of the input and output variables will be displayed and written to a file if the solver fails to converge.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src67", get_code("openmdao.test_suite.scripts.circuit_analysis.Circuit"), display=False)

:::{Admonition} `Circuit` class definition 
:class: dropdown

{glue:}`code_src67`
:::

In [ ]:
from packaging.version import Version
import numpy as np

import openmdao.api as om
from openmdao.test_suite.scripts.circuit_analysis import Circuit
from openmdao.utils.general_utils import printoptions

p = om.Problem()
model = p.model

model.add_subsystem('circuit', Circuit())

p.setup()

nl = model.circuit.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)

nl.options['iprint'] = 2
nl.options['debug_print'] = True
nl.options['err_on_non_converge'] = True

# set some poor initial guesses so that we don't converge
p.set_val('circuit.I_in', 0.1, units='A')
p.set_val('circuit.Vg', 0.0, units='V')
p.set_val('circuit.n1.V', 10.)
p.set_val('circuit.n2.V', 1e-3)

opts = {}
# formatting has changed in numpy 1.14 and beyond.
if Version(np.__version__) >= Version("1.14"):
    opts["legacy"] = '1.13'

with printoptions(**opts):
    # run the model
    try:
        p.run_model()
    except om.AnalysisError:
        pass

with open('solver_errors.0.out', 'r') as f:
    print(f.read())